Test model

In [1]:
import torch
from model import Model
from others.psnr import compute_psnr
from others.unet import UNet
from others.autoencoder import AutoEncoder
from others.rednet import REDNet
from torch.optim import Adam, SGD
from custom_model import MyModel

In [2]:
path_train = '../data/train_data.pkl'
path_val = '../data/val_data.pkl'
noisy_imgs_1, noisy_imgs_2 = torch.load(path_train, map_location=torch.device("cuda" if torch.cuda.is_available() else "cpu"))
noisy_imgs_1, noisy_imgs_2 = noisy_imgs_1.float(), noisy_imgs_2.float()
test, truth = torch.load(path_val, map_location=torch.device("cuda" if torch.cuda.is_available() else "cpu"))
test, truth = test.float(), truth.float()

c:\Users\stefa\anaconda3\lib\site-packages\torch\cuda\__init__.py:82: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at  C:\cb\pytorch_1000000000000\work\c10\cuda\CUDAFunctions.cpp:112.)
  return torch._C._cuda_getDeviceCount() > 0


In [3]:
def sample(tensor1, tensor2, k):
    perm = torch.randperm(tensor1.size(0))
    idx = perm[:k]
    return tensor1[idx], tensor2[idx]

## Training and testing

In [4]:
models = ["AutoEncoder"]
batch_sizes = [5]
features = [32]
epochs = 1
s1, s2 = sample(noisy_imgs_1, noisy_imgs_2, 1000)
t1, t2 = sample(test, truth, 1000)


In [5]:
network = AutoEncoder(num_features=32)
m = Model()
m.save_pretrained_model()

In [6]:
m.load_pretrained_model()

In [ ]:
def test_and_train(model, batch, features):
    network = UNet(num_features=features) if model == "UNet" else (REDNet(num_features=features) if model == "REDNet" else AutoEncoder(num_features=features))
    m = MyModel(network, batch)
    m.train(s1, s2, 10)
    return compute_psnr(m.predict(t1), t2), m

In [8]:
results = []
for model in models:
  for batch_size in batch_sizes:
      for f in features:
        error, m = test_and_train(model, batch_size, f)
        results.append(f'{model}_Layers5_Batch{batch_size}_Features{f}_Epochs10_Sample1000_psnr{error}')
        #torch.save(m.model.state_dict(), f'{model}_Layers10_Batch{batch_size}_Epochs10_Sample1000_{error}.pth')

0


KeyboardInterrupt: 

In [ ]:
results